In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
from scipy.stats import linregress

In [ ]:
# Kristina's code starts here


In [ ]:
# Kristina's code ends here

In [ ]:
# Jeff's code starts here

In [ ]:
# Jeff's code ends here

In [ ]:
# Faduma's code starts here

In [ ]:
# Faduma's code ends here

In [ ]:
# Emily's code starts here
storm_data = pd.read_csv('Resources/clean_storm_event_data.csv')
storm_data

In [ ]:
storm_data['Event Type'].unique()

# Counts of Tornado Eventsin Minnesota from 1950 to 2020

In [ ]:
year_list = storm_data["Year"].unique()
event_list = storm_data["Event Type"].unique()


clean_tornado_df = pd.DataFrame()
clean_tornado_df = storm_data[storm_data["Event Type"]=="Tornado"]


#clean_storm_data_mn_df
year_count_df = pd.DataFrame()
for i in year_list:
    year_count = clean_tornado_df[clean_tornado_df["Year"]==i]["Event Type"].count()
    year_count_df = year_count_df.append({"Year":i, "Count":year_count}, ignore_index=True)
year_count_df

x_data = (year_count_df["Year"].astype(int))
y_data = year_count_df["Count"].astype(float)

plt.scatter(x_data,y_data, s = 4)
plt.xlabel("Year")
plt.ylabel("Count")
plt.title("Counts of Tornado Events vs Year")

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_data,y_data)
regress_values_N = x_data * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,2))+", p="+str(round(pvalue_N,2))
min_x = x_data.min()
max_y = y_data.max()
plt.plot(x_data,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.show()

# Counts of Hail Events in Minnesota from 1950 to 2020

In [ ]:
clean_hail_df = pd.DataFrame()
clean_hail_df = storm_data[storm_data["Event Type"]=="Hail"]


#clean_storm_data_mn_df
year_count_df = pd.DataFrame()
for i in year_list:
    year_count = clean_hail_df[clean_hail_df["Year"]==i]["Event Type"].count()
    year_count_df = year_count_df.append({"Year":i, "Count":year_count}, ignore_index=True)
year_count_df

x_data = (year_count_df["Year"].astype(int))
y_data = year_count_df["Count"].astype(float)

plt.scatter(x_data,y_data, s = 4)
plt.xlabel("Year")
plt.ylabel("Count")
plt.title("Counts of Hail Events vs Year")

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_data,y_data)
regress_values_N = x_data * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,2))+", p="+str(round(pvalue_N,2))
min_x = x_data.min()
max_y = y_data.max()
plt.plot(x_data,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.show()

# Counts of Thunderstorm Wind Events in Minnesota from 1950 to 2020

In [ ]:
clean_thunderstorm_wind_df = pd.DataFrame()
clean_thunderstorm_wind_df = storm_data[storm_data["Event Type"]=="Thunderstorm Wind"]


#clean_storm_data_mn_df
year_count_df = pd.DataFrame()
for i in year_list:
    year_count = clean_thunderstorm_wind_df[clean_thunderstorm_wind_df["Year"]==i]["Event Type"].count()
    year_count_df = year_count_df.append({"Year":i, "Count":year_count}, ignore_index=True)
year_count_df

x_data = (year_count_df["Year"].astype(int))
y_data = year_count_df["Count"].astype(float)

plt.scatter(x_data,y_data, s = 4)
plt.xlabel("Year")
plt.ylabel("Count")
plt.title("Counts of Thunderstorm/Wind Events vs Year")

(slope_N, intercept_N, rvalue_N, pvalue_N, stderr_N) = linregress(x_data,y_data)
regress_values_N = x_data * slope_N + intercept_N
line_eq = "y = " + str(round(slope_N,2)) + "x + " + str(round(intercept_N,2))+", r = " +str(round(rvalue_N,2))+", p="+str(round(pvalue_N,2))
min_x = x_data.min()
max_y = y_data.max()
plt.plot(x_data,regress_values_N,"r-")
plt.annotate(line_eq,(min_x+1,max_y-1),fontsize=11,color="red") #writes to coordinates on graph
plt.show()

# Total Count of Tornado, Thunderstorm/Wind, and Hail Events in Minnesota from 1950 to 2020

In [ ]:



# storm_data.plot.bar(stacked=True)
# plt.show()

In [ ]:
# Emily's code ends here